# Tinder Analysis

# Mongo DB Data load

In [ ]:
import json
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='mongoadmin', password='pass1234')

# Get the list of DBs already defined
print(client.list_database_names())

# Create a new DB - bda
database = client['bda']

# Create a collection
tinder = database.tinder

# Load the data
with open('tinder_dataset/profiles_2021-11-10.json', 'r') as f:
    data = json.load(f)

['admin', 'bda', 'config', 'local']


In [ ]:
# Load the data
result = tinder.insert_many(data)
tinder.count_documents({})

In [ ]:
# Check if data was loaded correctly
tinder.find_one()

In [ ]:
cursor = tinder.find({"user.instagram": True})
list(cursor)[13]